In [1]:
## Dependencies
import pandas as pd

In [2]:
data_file = "purchase_data.json"
data_file_df = pd.read_json(data_file)
data_file_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [3]:
#Player Count
player_count = data_file_df["SN"].nunique()
player_count_df = pd.DataFrame({"Total Players": [player_count]})
player_count_df

,Total Players
0,573


In [4]:
#Purchase Analysis 
#Num of Unique Items
types = len(data_file_df["Item Name"].unique().tolist())

#Avg Purchase Price
avg_price = data_file_df["Price"].mean()

#Total Rev
total_rev = data_file_df["Price"].sum()

#Total Num of Purchases
total_purc = data_file_df["Item ID"].count()

#Create DataFrame
purchase_analysis = pd.DataFrame({"Number of Unique Items": [types], "Average Price": [avg_price], 
                                  "Total Revenue": [total_rev], "Total Purchases": [total_purc]})
#Format
purchase_analysis["Average Price"] = purchase_analysis["Average Price"].map("${:.2f}".format)
purchase_analysis["Total Revenue"] = purchase_analysis["Total Revenue"].map("${:,.2f}".format)

#Display Data
purchase_analysis

,Average Price,Number of Unique Items,Total Purchases,Total Revenue
0,$2.93,179,780,"$2,286.33"


In [5]:
#Gender Demographics 
gender_data = data_file_df["Gender"].value_counts()
play_perc = (gender_data/total_purc)*100

#Create new DataFrame
play_perc_df = pd.DataFrame({"Percent of Players": play_perc.round(2), "Total Count": gender_data})

#Display Data
play_perc_df

,Percent of Players,Total Count
Male,81.15,633
Female,17.44,136
Other / Non-Disclosed,1.41,11


In [6]:
#Gender Analysis
gender_ana = data_file_df.groupby(["Gender"])

gender_purc = gender_ana["Item ID"].count()
gender_avg = gender_ana["Price"].mean()
gender_rev = gender_ana["Price"].sum()
norm_total = (gender_rev / total_purc)

#Create DataFrame
gender_analysis = pd.DataFrame({"Purchase Count": gender_purc, "Average Price": gender_avg, 
                              "Total Revenue": gender_rev, "Normalized Data": norm_total.round(2)})

#Formate
gender_analysis["Average Price"] = gender_analysis["Average Price"].map("${:.2f}".format)
gender_analysis["Total Revenue"] = gender_analysis["Total Revenue"].map("${:,.2f}".format)

#Display
gender_analysis

,Average Price,Normalized Data,Purchase Count,Total Revenue
Gender,,,,
Female,$2.82,0.49,136,$382.91
Male,$2.95,2.39,633,"$1,867.68"
Other / Non-Disclosed,$3.25,0.05,11,$35.74


In [7]:
#Age Demographic 
bins = [0, 10, 14, 19, 23, 27, 31, 35, 39]
labels = ["<10", "10-14", "15-19", "19-23", "23-27", "27-31", "35-39", "40+"]

data_file_df["Age Demographic"] = pd.cut(data_file_df["Age"], bins, labels=labels)
data_file_df.head()

age_df = data_file_df.groupby(["Age Demographic"])

#Analysis
age_df_mean = age_df["Price"].mean()
age_df_sum = age_df["Price"].sum()
age_df_count = age_df["Item ID"].count()
norm_age = age_df_sum / total_purc

#Create new Dataframe
age_data = pd.DataFrame({"Avg. Purchase Price": age_df_mean,
                       "Total Purchase Value": age_df_sum,"Purchase Count": age_df_count,
                        "Normalized Total": norm_age.round(2)})

#Formate
age_data["Avg. Purchase Price"] = age_data["Avg. Purchase Price"].map("${:.2f}".format)
age_data["Total Purchase Value"] = age_data["Total Purchase Value"].map("${:.2f}".format)

#Display                                                                      
age_data

,Avg. Purchase Price,Normalized Total,Purchase Count,Total Purchase Value
Age Demographic,,,,
<10,$3.02,0.12,32,$96.62
10-14,$2.70,0.11,31,$83.79
15-19,$2.91,0.50,133,$386.42
19-23,$2.88,0.98,266,$765.31
23-27,$3.02,0.65,169,$510.02
27-31,$2.96,0.23,60,$177.40
35-39,$3.11,0.17,42,$130.64
40+,$2.75,0.11,30,$82.38


In [8]:
#Top Spenders
spender_total = data_file_df.groupby(['SN'])
spender_sum = pd.DataFrame(spender_total.sum()["Price"])
top_spenders = spender_sum.sort_values("Price", ascending=False)

#Spender Count & Mean
spender_count = data_file_df.groupby(['SN']).count()
spender_mean = spender_total["Price"].mean()

#Add Columns
top_spenders["Avg. Purchase Price"] = spender_mean
top_spenders["Purchase Count"] = spender_count["Item ID"]

#Format
top_spenders["Avg. Purchase Price"] = top_spenders["Avg. Purchase Price"].map("${:.2f}".format)
top_spenders["Price"] = top_spenders["Price"].map("${:.2f}".format)

#Display top 5
top_spenders.head(5)

,Price,Avg. Purchase Price,Purchase Count
SN,,,
Undirrala66,$17.06,$3.41,5
Saedue76,$13.56,$3.39,4
Mindimnya67,$12.74,$3.18,4
Haellysu29,$12.73,$4.24,3
Eoda93,$11.58,$3.86,3


In [31]:
#Most Popular
popular_total = data_file_df.groupby(['Item Name'])
popular_sum = pd.DataFrame(popular_total.count()["Item ID"])
most_pop = popular_sum.sort_values("Item ID", ascending=False)
org_most_pop = most_pop.rename(columns={"Item ID":"Purchase Count"})

total_pop_purc = popular_total["Price"].sum()

#Add Columns
item_total = data_file_df.groupby(["Item Name"])
item_count = pd.DataFrame(item_total.mean())
reduced_item_count = item_count.loc[:, ["Price"]]
id_count = item_count.loc[:, ["Item ID"]]

org_most_pop["Item id"] = item_count["Item ID"]
org_most_pop["Item Price"] = reduced_item_count["Price"]
org_most_pop["Total Purchase Value"] = total_pop_purc

#format
org_most_pop["Total Purchase Value"] = org_most_pop["Total Purchase Value"].map("${:.2f}".format)
org_most_pop["Item Price"] = org_most_pop["Item Price"].map("${:.2f}".format)
org_most_pop["Item id"] = org_most_pop["Item id"].map("{:.0f}".format)

#Display
org_most_pop.head(5)

,Purchase Count,Item id,Item Price,Total Purchase Value
Item Name,,,,
Final Critic,14,96,$2.76,$38.60
Arcane Gem,11,84,$2.23,$24.53
"Betrayal, Whisper of Grieving Widows",11,39,$2.35,$25.85
Stormcaller,10,105,$3.46,$34.65
Woeful Adamantite Claymore,9,175,$1.24,$11.16


In [38]:
#Most profitable
prof_total = data_file_df.groupby(['Item Name'])
prof_sum = pd.DataFrame(prof_total.sum()["Price"])
most_prof = prof_sum.sort_values("Price", ascending=False)

#Rename Columns
org_most_prof = most_prof.rename(columns={"Price":"Total Purchase Value"})

#Add Columns
org_most_prof["Purchase Count"] = prof_total["Item Name"].count()
org_most_prof["Item Price"] = reduced_item_count["Price"]
org_most_prof["Item ID"] = item_count["Item ID"]

#Format
org_most_prof["Total Purchase Value"] = org_most_prof["Total Purchase Value"].map("${:.2f}".format)
org_most_prof["Item Price"] = org_most_prof["Item Price"].map("${:.2f}".format)
org_most_prof["Item ID"] = org_most_prof["Item ID"].map("{:.0f}".format)

#Display
org_most_prof.head(5)

,Total Purchase Value,Purchase Count,Item Price,Item ID
Item Name,,,,
Final Critic,$38.60,14,$2.76,96
Retribution Axe,$37.26,9,$4.14,34
Stormcaller,$34.65,10,$3.46,105
Spectral Diamond Doomblade,$29.75,7,$4.25,115
Orenmir,$29.70,6,$4.95,32
